In [ ]:
initial_filter = 2048
depth = 30
def model():
    def conv1d(_input, filters, kernel_size=4, padding='same', activation='relu', pooling=True):
        if pooling:
            d = Conv1D(filters, kernel_size=kernel_size, strides=2, padding=padding)(_input)
        else:
            d = Conv1D(filters, kernel_size=kernel_size, strides=1, padding=padding)(_input)
        if activation == 'relu':
            d = Activation('relu')(d)
        elif activation == 'leaky_relu':
            d = LeakyReLU(0.2)(d)
        d = BatchNormalization()(d)
        ret = d
        return ret
    def upconv1d(_input, filters, kernel_size=4, strides=2, padding='same', activation='relu'):
        u = Conv1DTranspose(filters, kernel_size=kernel_size, strides=strides, padding=padding)(_input)
        if activation == 'relu':
            u = Activation('relu')(u)
        elif activation == 'leaky_relu':
            u = LeakyReLU(0.2)(u)
        u = BatchNormalization()(u)
        ret = u
        return ret
    
    _input = Input(shape=shape)
    
    net = wavenet_convolutional_layers(filters=64, kernel_size=4, depth=depth)(_input)
    
    # Downsample
    net = conv1d(net, filters=initial_filter/2, activation='leaky_relu')
    net = conv1d(net, filters=initial_filter, activation='leaky_relu')
    net = conv1d(net, filters=initial_filter*2, activation='leaky_relu')
    net = conv1d(net, filters=initial_filter*4, activation='leaky_relu')
    # Upsample
    net = upconv1d(net, filters=initial_filter*2, activation='relu')
    net = upconv1d(net, filters=initial_filter, activation='relu')
    net = upconv1d(net, filters=initial_filter/2, activation='relu')
    net = Conv1DTranspose(filters=1, kernel_size=4, strides=2, padding='same', activation='tanh')(net)
    
    net = wavenet_convolutional_layers(filters=64, kernel_size=4, depth=depth)(net)
    net = Conv1D(2048, 3, padding='same')(net)
    net = Conv1D(256, 3, padding='same')(net)
    out = Conv1D(1, 1, padding='same')(net)
    model = Model(_input, out)
    model.summary()
    return model